## Încapsulări

În exemplul anterior, am lucrat cu vectori de tip bag-of-words de dimensiuni mari, având lungimea `vocab_size`, și am convertit explicit vectorii de reprezentare pozițională de dimensiuni mici în reprezentări sparse de tip one-hot. Această reprezentare one-hot nu este eficientă din punct de vedere al memoriei. În plus, fiecare cuvânt este tratat independent de celelalte, astfel încât vectorii codificați one-hot nu exprimă similitudini semantice între cuvinte.

În această unitate, vom continua să explorăm setul de date **News AG**. Pentru început, să încărcăm datele și să obținem câteva definiții din unitatea anterioară.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Ce este un embedding?

Ideea de **embedding** este de a reprezenta cuvintele folosind vectori densi de dimensiuni mai mici, care reflectă semnificația semantică a cuvântului. Vom discuta mai târziu cum să construim embedding-uri semnificative pentru cuvinte, dar deocamdată să ne gândim la embedding-uri ca la o modalitate de a reduce dimensiunea unui vector de cuvinte.

Astfel, un strat de embedding primește un cuvânt ca intrare și produce un vector de ieșire cu dimensiunea specificată `embedding_size`. Într-un fel, este foarte similar cu un strat `Dense`, dar în loc să primească un vector codificat one-hot ca intrare, poate primi un număr corespunzător cuvântului.

Folosind un strat de embedding ca prim strat în rețeaua noastră, putem trece de la modelul bag-of-words la un model **embedding bag**, unde mai întâi convertim fiecare cuvânt din textul nostru în embedding-ul corespunzător, iar apoi calculăm o funcție agregată pentru toate aceste embedding-uri, cum ar fi `sum`, `average` sau `max`.

![Imagine care arată un clasificator embedding pentru cinci cuvinte dintr-o secvență.](../../../../../translated_images/ro/embedding-classifier-example.b77f021a7ee67eee.webp)

Rețeaua noastră neurală de clasificare constă din următoarele straturi:

* Strat `TextVectorization`, care primește un șir de caractere ca intrare și produce un tensor de numere de tokeni. Vom specifica o dimensiune rezonabilă pentru vocabular, `vocab_size`, și vom ignora cuvintele utilizate mai rar. Forma de intrare va fi 1, iar forma de ieșire va fi $n$, deoarece vom obține $n$ tokeni ca rezultat, fiecare dintre ei conținând numere de la 0 la `vocab_size`.
* Strat `Embedding`, care primește $n$ numere și reduce fiecare număr la un vector dens de o lungime dată (100 în exemplul nostru). Astfel, tensorul de intrare cu forma $n$ va fi transformat într-un tensor $n\times 100$.
* Strat de agregare, care calculează media acestui tensor de-a lungul primei axe, adică va calcula media tuturor celor $n$ tensori de intrare corespunzători diferitelor cuvinte. Pentru a implementa acest strat, vom folosi un strat `Lambda` și vom transmite funcția pentru a calcula media. Ieșirea va avea forma 100 și va fi reprezentarea numerică a întregii secvențe de intrare.
* Clasificator final `Dense` liniar.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


În rezumatul `summary`, în coloana **output shape**, prima dimensiune a tensorului `None` corespunde dimensiunii minibatch-ului, iar a doua corespunde lungimii secvenței de tokeni. Toate secvențele de tokeni din minibatch au lungimi diferite. Vom discuta cum să gestionăm acest aspect în secțiunea următoare.

Acum să antrenăm rețeaua:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Notă** că construim vectorizatorul pe baza unui subset de date. Acest lucru se face pentru a accelera procesul și poate duce la o situație în care nu toți tokenii din textul nostru sunt prezenți în vocabular. În acest caz, acei tokeni vor fi ignorați, ceea ce poate duce la o acuratețe ușor mai scăzută. Totuși, în viața reală, un subset de text oferă adesea o estimare bună a vocabularului.


### Gestionarea dimensiunilor variabile ale secvențelor

Să înțelegem cum are loc antrenarea în minibatch-uri. În exemplul de mai sus, tensorul de intrare are dimensiunea 1, iar noi folosim minibatch-uri de lungime 128, astfel încât dimensiunea reală a tensorului este $128 \times 1$. Totuși, numărul de tokeni din fiecare propoziție este diferit. Dacă aplicăm stratul `TextVectorization` unui singur input, numărul de tokeni returnați este diferit, în funcție de modul în care textul este tokenizat:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Cu toate acestea, atunci când aplicăm vectorizatorul la mai multe secvențe, acesta trebuie să producă un tensor de formă dreptunghiulară, astfel încât să umple elementele neutilizate cu tokenul PAD (care în cazul nostru este zero):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Aici putem vedea încorporările:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Notă**: Pentru a minimiza cantitatea de umplere, în unele cazuri are sens să se sorteze toate secvențele din setul de date în ordinea lungimii crescătoare (sau, mai precis, numărul de tokeni). Acest lucru va asigura că fiecare minibatch conține secvențe de lungime similară.


## Încapsulări semantice: Word2Vec

În exemplul anterior, stratul de încapsulare a învățat să transforme cuvintele în reprezentări vectoriale, însă aceste reprezentări nu aveau o semnificație semantică. Ar fi ideal să învățăm o reprezentare vectorială astfel încât cuvintele similare sau sinonime să corespundă unor vectori apropiați între ei în funcție de o anumită distanță vectorială (de exemplu, distanța euclidiană).

Pentru a realiza acest lucru, trebuie să preantrenăm modelul de încapsulare pe o colecție mare de texte utilizând o tehnică precum [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Aceasta se bazează pe două arhitecturi principale folosite pentru a produce o reprezentare distribuită a cuvintelor:

 - **Continuous bag-of-words** (CBoW), unde antrenăm modelul să prezică un cuvânt pe baza contextului din jur. Având ngram-ul $(W_{-2},W_{-1},W_0,W_1,W_2)$, scopul modelului este să prezică $W_0$ pe baza $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** este opusul lui CBoW. Modelul folosește fereastra de cuvinte contextuale din jur pentru a prezice cuvântul curent.

CBoW este mai rapid, iar skip-gram, deși mai lent, face o treabă mai bună în reprezentarea cuvintelor rare.

![Imagine care ilustrează algoritmii CBoW și Skip-Gram pentru conversia cuvintelor în vectori.](../../../../../translated_images/ro/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Pentru a experimenta cu încapsularea Word2Vec preantrenată pe setul de date Google News, putem folosi biblioteca **gensim**. Mai jos găsim cuvintele cele mai similare cu 'neural'.

> **Notă:** Când creați pentru prima dată vectori de cuvinte, descărcarea acestora poate dura ceva timp!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Putem, de asemenea, să extragem încorporarea vectorială din cuvânt, pentru a fi utilizată în antrenarea modelului de clasificare. Încorporarea are 300 de componente, dar aici arătăm doar primele 20 de componente ale vectorului pentru claritate:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Lucrul grozav despre încorporările semantice este că poți manipula codificarea vectorială pe baza semanticii. De exemplu, putem cere să găsim un cuvânt a cărui reprezentare vectorială este cât mai aproape posibil de cuvintele *rege* și *femeie*, și cât mai departe posibil de cuvântul *bărbat*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Un exemplu de mai sus folosește ceva magie internă GenSym, dar logica de bază este de fapt destul de simplă. Un lucru interesant despre încorporări este că poți efectua operații normale pe vectori de încorporare, iar acestea ar reflecta operații asupra **semnificațiilor** cuvintelor. Exemplul de mai sus poate fi exprimat în termeni de operații pe vectori: calculăm vectorul corespunzător **REGE-BĂRBAT+FEMEIE** (operațiile `+` și `-` sunt efectuate pe reprezentările vectoriale ale cuvintelor corespunzătoare), și apoi găsim cel mai apropiat cuvânt din dicționar de acel vector:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTĂ**: A trebuit să adăugăm niște coeficienți mici vectorilor *man* și *woman* - încearcă să îi elimini pentru a vedea ce se întâmplă.

Pentru a găsi vectorul cel mai apropiat, folosim mecanismele TensorFlow pentru a calcula un vector de distanțe între vectorul nostru și toți vectorii din vocabular, iar apoi găsim indexul cuvântului minim folosind `argmin`.


Deși Word2Vec pare o metodă excelentă pentru a exprima semantica cuvintelor, are multe dezavantaje, inclusiv următoarele:

* Atât modelele CBoW, cât și skip-gram sunt **embedding-uri predictive**, și iau în considerare doar contextul local. Word2Vec nu profită de contextul global.
* Word2Vec nu ia în considerare **morfologia** cuvintelor, adică faptul că sensul unui cuvânt poate depinde de diferite părți ale cuvântului, cum ar fi rădăcina.

**FastText** încearcă să depășească a doua limitare și se bazează pe Word2Vec prin învățarea reprezentărilor vectoriale pentru fiecare cuvânt și pentru n-gramele de caractere găsite în interiorul fiecărui cuvânt. Valorile reprezentărilor sunt apoi mediate într-un singur vector la fiecare pas de antrenament. Deși acest lucru adaugă multă calculare suplimentară în etapa de pre-antrenare, permite embedding-urilor să encodeze informații despre sub-cuvinte.

O altă metodă, **GloVe**, folosește o abordare diferită pentru embedding-urile de cuvinte, bazată pe factorizarea matricei cuvânt-context. Mai întâi, construiește o matrice mare care numără de câte ori apar cuvintele în diferite contexte, iar apoi încearcă să reprezinte această matrice în dimensiuni mai mici într-un mod care minimizează pierderea de reconstrucție.

Biblioteca gensim suportă aceste embedding-uri de cuvinte, și poți experimenta cu ele schimbând codul de încărcare a modelului de mai sus.


## Utilizarea embedding-urilor pre-antrenate în Keras

Putem modifica exemplul de mai sus pentru a prepopula matricea din stratul nostru de embedding cu embedding-uri semantice, cum ar fi Word2Vec. Vocabularul embedding-urilor pre-antrenate și cel al corpusului de text probabil nu se vor potrivi, așa că trebuie să alegem unul. Aici explorăm cele două opțiuni posibile: utilizarea vocabularului tokenizer-ului și utilizarea vocabularului embedding-urilor Word2Vec.

### Utilizarea vocabularului tokenizer-ului

Când folosim vocabularul tokenizer-ului, unele dintre cuvintele din vocabular vor avea embedding-uri Word2Vec corespunzătoare, iar altele vor lipsi. Având în vedere că dimensiunea vocabularului nostru este `vocab_size`, iar lungimea vectorului embedding Word2Vec este `embed_size`, stratul de embedding va fi reprezentat de o matrice de greutăți cu forma `vocab_size`$\times$`embed_size`. Vom popula această matrice parcurgând vocabularul:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Pentru cuvintele care nu sunt prezente în vocabularul Word2Vec, putem fie să le lăsăm ca zerouri, fie să generăm un vector aleatoriu.

Acum putem defini un strat de embedding cu greutăți pre-antrenate:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Notă**: Observați că am setat `trainable=False` atunci când am creat `Embedding`, ceea ce înseamnă că nu reantrenăm stratul Embedding. Acest lucru poate duce la o acuratețe ușor mai scăzută, dar accelerează procesul de antrenare.

### Utilizarea vocabularului embedding

O problemă cu abordarea anterioară este că vocabularele utilizate în TextVectorization și Embedding sunt diferite. Pentru a depăși această problemă, putem folosi una dintre următoarele soluții:
* Reantrenarea modelului Word2Vec pe vocabularul nostru.
* Încărcarea dataset-ului nostru folosind vocabularul din modelul Word2Vec preantrenat. Vocabularele utilizate pentru încărcarea dataset-ului pot fi specificate în timpul încărcării.

A doua abordare pare mai simplă, așa că haideți să o implementăm. În primul rând, vom crea un strat `TextVectorization` cu vocabularul specificat, preluat din embedding-urile Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Biblioteca de încorporări de cuvinte gensim conține o funcție convenabilă, `get_keras_embeddings`, care va crea automat stratul de încorporări Keras corespunzător pentru tine.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Unul dintre motivele pentru care nu observăm o acuratețe mai mare este faptul că unele cuvinte din setul nostru de date lipsesc din vocabularul GloVe preantrenat și, astfel, sunt practic ignorate. Pentru a depăși acest obstacol, putem antrena propriile noastre încorporări bazate pe setul nostru de date.


## Încapsulări contextuale

O limitare importantă a reprezentărilor tradiționale de încapsulare pre-antrenate, cum ar fi Word2Vec, este faptul că, deși pot surprinde o parte din sensul unui cuvânt, nu pot face diferența între sensurile diferite. Acest lucru poate cauza probleme în modelele ulterioare.

De exemplu, cuvântul „play” are sensuri diferite în aceste două propoziții:
- Am fost la o **piesă** de teatru.
- John vrea să se **joace** cu prietenii săi.

Încapsulările pre-antrenate despre care am vorbit reprezintă ambele sensuri ale cuvântului „play” în aceeași încapsulare. Pentru a depăși această limitare, trebuie să construim încapsulări bazate pe **modelul lingvistic**, care este antrenat pe un corpus mare de text și *știe* cum pot fi utilizate cuvintele în contexte diferite. Discutarea încapsulărilor contextuale depășește scopul acestui tutorial, dar vom reveni la ele când vom vorbi despre modelele lingvistice în unitatea următoare.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
